In [1]:
import tensorflow as tf
import numpy as np

In [2]:
dtype = 'float32'
tf.keras.backend.set_floatx(dtype)

In [3]:
# fashion_mnist = tf.keras.datasets.fashion_mnist
# (X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

# X_train = X_train.astype(dtype) / 255.0
# y_train = y_train.astype(dtype)
# X_test = X_test.astype(dtype)  / 255.0
# y_test = y_test.astype(dtype)

# X_train = np.reshape(X_train, (-1, 784))
# X_test = np.reshape(X_test, (-1, 784))

In [4]:
cifar10 = tf.keras.datasets.cifar10
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

X_train = X_train.astype(dtype) / 255.0
y_train = y_train.astype(dtype)
X_test = X_test.astype(dtype)  / 255.0
y_test = y_test.astype(dtype)

X_train = np.reshape(X_train, (-1, 3072))
X_test = np.reshape(X_test, (-1, 3072))

In [82]:
class SSRegularizer(tf.keras.regularizers.Regularizer):
    def __init__(self, l1):
        self.l1 = l1

    def __call__(self, x):
        scaling_vector = tf.cumsum(tf.constant(self.l1, shape=(x.shape[0],), dtype=dtype), axis=0) - self.l1
        return tf.reduce_sum(tf.reshape(scaling_vector, (-1, 1)) * tf.abs(x))

    def get_config(self):
        return {'l1': float(self.l1)}


class SSLayer(tf.keras.Model):
    def __init__(self, input_units, units, activation, l1, kernel_initializer, bias_initializer, regularize=True):
        super().__init__()

        self.input_units = input_units
        self.units = units
        self.activation = activation
        self.l1 = l1
        self.kernel_initializer = kernel_initializer
        self.bias_initializer = bias_initializer
        
        self.A = tf.keras.activations.get(activation)
        self.W_init = tf.keras.initializers.get(kernel_initializer)
        self.b_init = tf.keras.initializers.get(bias_initializer)
        self.regularizer = SSRegularizer(self.l1)
        
        self.W = tf.Variable(
            name='W',
            initial_value=self.W_init(shape=(input_units, units), dtype=dtype),
            trainable=True)
        
        self.b = tf.Variable(
            name='b',
            initial_value=self.b_init(shape=(units,), dtype=dtype),
            trainable=True)
        
        if regularize:
            self.add_loss(lambda: self.regularizer(self.W))
            self.add_loss(lambda: self.regularizer(self.b))
    
    def call(self, inputs):
        return self.A(tf.matmul(inputs, self.W) + self.b)
    
    def copy_without_regularization(self):
        copy = SSLayer(self.input_units, self.units, self.activation, self.l1, self.kernel_initializer, self.bias_initializer, regularize=False)
        copy.W = self.W
        copy.b = self.b
        return copy


class SSModel(tf.keras.Model):
    def __init__(self, layer_sizes, activation=None, l1=0.01, kernel_initializer='glorot_uniform', bias_initializer='zeros'):
        super().__init__()
        
        self.sslayers = list()
        for l in range(len(layer_sizes) - 1):
            input_units = layer_sizes[l]
            units = layer_sizes[l + 1]
            if l == 0:  # First layer
                layer = SSLayer(input_units, units, activation, 0., kernel_initializer, bias_initializer)
            elif l == len(layer_sizes) - 2:  # Last layer
                layer = SSLayer(input_units, units, 'softmax', l1, kernel_initializer, bias_initializer)
            else:  
                layer = SSLayer(input_units, units, activation, l1, kernel_initializer, bias_initializer)
            self.sslayers.append(layer)

    def call(self, inputs):
        x = inputs
        for layer in self.sslayers:
            x = layer(x)
        return x
    
    def get_layer_sizes(self):
        layer_sizes = list()
        for l in range(len(self.sslayers)):
            layer = self.sslayers[l]
            layer_sizes.append(layer.W.shape[0])
            if l == len(self.sslayers) - 1:  # Last layer
                layer_sizes.append(layer.W.shape[1])
        return layer_sizes
    
    def print_neurons(self):
        for layer in self.sslayers[1:]:
            print(get_param_string(layer.W))
    
    def remove_regularization(self):
        for l in range(len(self.sslayers)):
            self.sslayers[l] = self.sslayers[l].copy_without_regularization()
        
    def prune(self, threshold=0.001):
        for l in range(len(self.sslayers) - 1):
            layer1 = self.sslayers[l]
            layer2 = self.sslayers[l + 1]
            
            W1 = layer1.W.value()
            b1 = layer1.b.value()
            W2 = layer2.W.value()

            neurons_are_active = tf.math.reduce_max(tf.abs(W2), axis=1) >= threshold
            active_neurons_indices = tf.reshape(tf.where(neurons_are_active), (-1,))
            
            new_W1 = tf.gather(W1, active_neurons_indices, axis=1)
            new_b1 = tf.gather(b1, active_neurons_indices, axis=0)
            new_W2 = tf.gather(W2, active_neurons_indices, axis=0)

            layer1.W = tf.Variable(name='W', initial_value=new_W1, trainable=True)
            layer1.b = tf.Variable(name='b', initial_value=new_b1, trainable=True)
            layer2.W = tf.Variable(name='W', initial_value=new_W2, trainable=True)
    
    def grow(self, min_new_neurons=5, scaling_factor=0.001):   
        for l in range(len(self.sslayers) - 1):
            layer1 = self.sslayers[l]
            layer2 = self.sslayers[l + 1]
       
            W1 = layer1.W.value()
            b1 = layer1.b.value()
            W2 = layer2.W.value()

            n_new_neurons = max(min_new_neurons, int(W1.shape[1] * 0.2))

            W1_growth = layer1.W_init(shape=(W1.shape[0], W1.shape[1] + n_new_neurons), dtype=dtype)[:, -n_new_neurons:] * scaling_factor
            b1_growth = layer1.b_init(shape=(n_new_neurons,), dtype=dtype)
            W2_growth = layer2.W_init(shape=(W2.shape[0] + n_new_neurons, W2.shape[1]), dtype=dtype)[-n_new_neurons:, :]

            new_W1 = tf.concat([W1, W1_growth], axis=1)
            new_b1 = tf.concat([b1, b1_growth], axis=0)
            new_W2 = tf.concat([W2, W2_growth], axis=0)

            layer1.W = tf.Variable(name='W1', initial_value=new_W1, trainable=True)
            layer1.b = tf.Variable(name='b1', initial_value=new_b1, trainable=True)
            layer2.W = tf.Variable(name='W2', initial_value=new_W2, trainable=True)

In [87]:
def get_param_string(weights):
    param_string = ""
    max_parameters = tf.math.reduce_max(tf.abs(weights), axis=1).numpy()
    magnitudes = np.floor(np.log10(max_parameters))
    for m in magnitudes:
        if m > 0:
            m = 0
        param_string += str(int(-m))
    return param_string


def print_epoch_statistics(model):
    y_pred = model(X_train)
    loss = tf.reduce_mean(tf.keras.losses.sparse_categorical_crossentropy(y_train, y_pred))
    accuracy = tf.reduce_mean(tf.keras.metrics.sparse_categorical_accuracy(y_train, y_pred))
    
    y_pred_val = model(X_test)
    val_loss = tf.reduce_mean(tf.keras.losses.sparse_categorical_crossentropy(y_test, y_pred_val))
    val_accuracy = tf.reduce_mean(tf.keras.metrics.sparse_categorical_accuracy(y_test, y_pred_val))
    print(f"loss: {loss} - accuracy: {accuracy} - val_loss: {val_loss} - val_accuracy: {val_accuracy}")
    print(f"layer sizes: {model.get_layer_sizes()}")
    model.print_neurons()


def train_model(model, optimizer, epochs, batch_size, train_dataset):
    for epoch in range(epochs):
        print(f"Epoch {epoch + 1}/{epochs}")
        
        print("Before growing:")
        print_epoch_statistics(model)
        model.grow(min_new_neurons=10, scaling_factor=0.001)
        print("After growing:")
        print_epoch_statistics(model)

        for step, (x_batch, y_batch) in enumerate(train_dataset):
            with tf.GradientTape() as tape:
                y_pred = model(x_batch, training=True)
                loss_value = tf.reduce_mean(tf.keras.losses.sparse_categorical_crossentropy(y_batch, y_pred))
                loss_value += sum(model.losses)

            grads = tape.gradient(loss_value, model.trainable_variables)
            optimizer.apply_gradients(zip(grads, model.trainable_variables))
        
        print("Before pruning:")
        print_epoch_statistics(model)
        model.prune(threshold=0.001)
        print("After pruning:")
        print_epoch_statistics(model)

In [88]:
epochs = 20
batch_size = 32

train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)

model = SSModel(layer_sizes=[3072, 100, 100, 10], activation='relu', l1=0.0001, kernel_initializer='he_normal')
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

train_model(model, optimizer, epochs, batch_size, train_dataset)

Epoch 1/20
Before growing:
loss: 2.443971633911133 - accuracy: 0.10277999937534332 - val_loss: 2.4407873153686523 - val_accuracy: 0.10570000112056732
layer sizes: [3072, 100, 100, 10]
1111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111
1111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111
After growing:
loss: 2.4439027309417725 - accuracy: 0.10270000249147415 - val_loss: 2.4407172203063965 - val_accuracy: 0.10570000112056732
layer sizes: [3072, 120, 120, 10]
111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111
111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111
Before pruning:
loss: 1.7542556524276733 - accuracy: 0.3724200129508972 - val_loss: 1.7534072399139404 - val_accuracy: 0.37279999256134033
layer sizes: [3072, 120, 120, 10]
12144444443444244414144444

KeyboardInterrupt: 